In [ ]:
import findspark
findspark.init()

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, desc, count, explode, split, regexp_replace, collect_list, array_sort, reverse, unix_timestamp, row_number,
    when, lit, lead, avg, udf
)

from pyspark.sql import Window

from pyspark.sql.types import ArrayType, StructType, StructField, StringType, LongType, TimestampType, DoubleType, IntegerType

import os

In [ ]:
spark = (
    SparkSession
    .builder
    .appName('UDFs I')
    .getOrCreate()
)

# Task I

* convert question tags to an array using UDFs

In [ ]:
base_path = os.getcwd()

project_path = ('/').join(base_path.split('/')[0:-2]) 

data_input_path = os.path.join(project_path, 'data/questions-json')

In [ ]:
questionsDF = (
    spark
    .read
    .format('json')
    .option('path', data_input_path)
    .load()
)

In [ ]:
questionsDF.select('tags').show(truncate=False)

In [ ]:
@udf(ArrayType(StringType()))
def transform_tags(tags):
    tags_list = tags.split('><')
    tags_list[0] = tags_list[0][1:]
    tags_list[-1] = tags_list[-1][0:-1]
    return tags_list

In [ ]:
(
    questionsDF
    .select('tags')
    .withColumn('tags_arr', transform_tags('tags'))
).show(truncate=40)